In [1]:
import graphviz
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

from groundupml.supervised.random_forest import RandomForest

%matplotlib inline

In [2]:
# Demonstrate forest can learn XOR function
np.random.seed(2)  # Set seed for reproducibility
X_train = np.array([[0, 0],
                    [1, 0],
                    [0, 1],
                    [1, 1],
                    [0, 0]])

y_train = np.array([0, 1, 1, 0, 0])

rf = RandomForest(n_trees=3, max_depth=5, impurity_func='gini')
rf.fit(X_train, y_train)

for tree in rf.trees:
    print(tree)

└────f0 <= 0
      └──T──0
      └──F──f1 <= 0
            └──T──1
            └──F──0

└────f0 <= 0
      ├──T──f1 <= 0
      │      └──T──0
      │      └──F──1
      └──F──f1 <= 0
            └──T──1
            └──F──0

└────f0 <= 0
      ├──T──f1 <= 0
      │      └──T──0
      │      └──F──1
      └──F──0



In [3]:
#def plot_tree(tree):
#        G = nx.DiGraph()
#        nodes = np.arange(0, 8).tolist()
#        G.add_nodes_from(nodes)
#        G.add_edges_from([(0, 1), (0, 2), (1, 3), (1, 4), (2, 5), (2, 6), (2, 7)])
#
#        pos = {0:(10, 10),
#                1:(7.5, 7.5),
#                2:(12.5, 7.5),
#                3:(6, 6),
#                4:(9, 6),
#                5:(11, 6),
#                6:(14, 6),
#                7:(17, 6)}
#
#        labels = {0:'CEO', 1:'VP1', 2:'VP2', 3:'StaffA', 4:'StaffB', 5:'StaffC',
#                6:'StaffD', 7:'StaffE'}
#
#        nx.draw_networkx(G, pos=pos, labels=labels, arrows=True, node_shape='s',
#                        node_size=1000, node_color='w', font_size=10)
#
#        plt.title('Decision Tree')
#
#plot_tree(rf.trees[0])

In [4]:
#def plot_tree(tree, feature_names=None, class_names=None, node_ids=False,
#              ax=None, fontsize=12):
#    """Plots the learned tree model. Only works for binary classification.
#    Args:
#        tree (DecisionTree): The learned decision tree model to plot.
#        feature_names (list): The names of the features.
#        class_names (list): The names of the classes.
#        node_ids (bool): Whether to display node ids.
#        proportion (bool): Whether to display fraction of samples in each class.
#        filled (bool): Whether to fill node boxes with colors.
#        fontsize (int): Fontsize for text.
#    """
#    def _plot_node(node, depth):
#        if node.children == []:
#            # Leaf Node
#            ax.text(x=depth, y=0.5, s=node.value, va='center', ha='center')
#            return max_depth
#        else:
#            # Non-leaf Node
#            feature_name = feature_names[node.feature_index] if feature_names else \
#                node.feature_index
#            threshold = node.threshold
#            ax.text(x=depth, y=0.5, 
#                    s=f'{feature_name} <= {threshold:.2f}',
#                    va='center', ha='center')
#
#            # Traverse the branches
#            left_branch, right_branch = node.children
#            max_depth_l = _plot_node(left_branch, depth=depth+1, max_depth=max_depth)
#            max_depth_r = _plot_node(right_branch, depth=depth+1, max_depth=max_depth)
#
#            return np.max([max_depth_l, max_depth_r])
#
#
#
#    max_depth = _plot_node(tree.root, depth=1, max_depth=1)
#
#
#plot_tree(rf.trees[0])

In [10]:

def export_graphviz(tree, feature_names, class_names):
    dot = graphviz.Digraph()

    def add_node(node):
        # Get the node ID and label
        node_id = str(id(node))
        if node.value is not None:
            # Leaf Node
            node_label = node.value
            print('adding leaf node')
            dot.node(node_id)#, label=node_label)
        else:
            print('NON-LEAF')
            # Non-leaf Node
            node_label = str(node.feature_index)
            dot.node(node_id, label=node_label)
            left_child, right_child = node.children

            # Build children
            dot.edge(node_id, str(id(left_child)), label='True')
            dot.edge(node_id, str(id(right_child)), label='False')
            add_node(left_child)
            add_node(right_child)
        
    add_node(tree.root)

    return dot.source


print(export_graphviz(rf.trees[0], feature_names=None, class_names=None))


NON-LEAF
adding leaf node


TypeError: cannot use a string pattern on a bytes-like object

In [ ]:
X_test = X_train
predictions = rf.predict(X_test)
print(predictions)